# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sqlite3
import pandas as pd
import re
from sqlalchemy import create_engine
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
#ML Pipelines
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
import pickle
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# load data from database
engine = create_engine('sqlite:///disaster.db')
df = pd.read_sql_table('disaster', engine)
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

In [58]:
 df.groupby('genre').count()

,id,message,original,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
genre,,,,,,,,,,,,,,,,,,,,,
direct,10766,10766,10170,10766,10766,10766,10766,10766,10766,10766,...,10766,10766,10766,10766,10766,10766,10766,10766,10766,10766
news,13054,13054,0,13054,13054,13054,13054,13054,13054,13054,...,13054,13054,13054,13054,13054,13054,13054,13054,13054,13054
social,2396,2396,0,2396,2396,2396,2396,2396,2396,2396,...,2396,2396,2396,2396,2396,2396,2396,2396,2396,2396


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """
    Normalize, tokenize and stem text into words
    
    Args:
    text, a string of words 
       
    Returns:
    stem, array of strings containing words
    """
    #lower case and remove special punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ",text.lower())
    #split using tokenizer
    words = word_tokenize(text)
    #remove stopwords to reduce vocab & use stem
    words = [w for w in words if w not in stopwords.words("english")]
    
    return words

In [4]:
#test first line of X to see if it has tokenized the words correctly
print(X[0])
tokenize(X[0])

Weather update - a cold front from Cuba that could pass over Haiti


['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()), 
    ('clf', MultiOutputClassifier(RandomForestClassifier())),
])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=10)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def pred_loop(actual, predicted, col_names):
    
    """
    Args:
    actual: Array with labels
    predicted: Array with labels
    col_names: Names for each column
       
    Returns:
    predictions_df: Dataframe with recall, precision, f1 and accuracy scores
    """
    metrics = []
    
    #Loop to score each of the metrics and predicitions for inputted arrays
    for i in range(len(col_names)):
        accuracy = accuracy_score(actual[:, i], predicted[:, i])
        precision = precision_score(actual[:, i], predicted[:, i], average='micro')
        recall = recall_score(actual[:, i], predicted[:, i], average='micro')
        f1 = f1_score(actual[:, i], predicted[:, i], average='micro')
        
        metrics.append([accuracy, precision, recall, f1])
    
    #Dataframe creation containing the predictions
    metrics = np.array(metrics)
    predictions_df = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return predictions_df

In [8]:
#model and pipeline run on the training set
y_pred = pipeline.predict(X_train)
col_names = list(y.columns.values)
Xtrain_pred = pred_loop(np.array(y_train), y_pred, col_names)

In [9]:
Xtrain_pred

,Accuracy,Precision,Recall,F1
related,0.989625,0.989625,0.989625,0.989625
request,0.988048,0.988048,0.988048,0.988048
offer,0.998830,0.998830,0.998830,0.998830
aid_related,0.986624,0.986624,0.986624,0.986624
medical_help,0.988353,0.988353,0.988353,0.988353
medical_products,0.990133,0.990133,0.990133,0.990133
search_and_rescue,0.993643,0.993643,0.993643,0.993643
security,0.994812,0.994812,0.994812,0.994812
military,0.995931,0.995931,0.995931,0.995931
child_alone,1.000000,1.000000,1.000000,1.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
#to do this we need to tune the hyperparameters
pipeline.get_params

<bound method Pipeline.get_params of Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])>

In [11]:
#hyper-parameter selection 
parameters = {
    #'vect__min_df': [1, 5],
    #'vect__max_features': [10000],
    #'clf__estimator__n_estimators': [300],
    'tfidf__smooth_idf':[True, False],
    #'clf__estimator__n_estimators':[10, 25], 
    'clf__estimator__min_samples_split':[2, 5, 10]
}

cv = GridSearchCV(pipeline, parameters, cv=3, n_jobs=-1)
cv

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__smooth_idf': [True, False], 'clf__estimator__min_samples_split': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
model_impro = cv.fit(X_train, y_train)

In [13]:
model_impro_predict = model_impro.predict(X_train)

In [14]:
predictor_model = pred_loop(np.array(y_train), model_impro_predict, col_names)

In [15]:
predictor_model

,Accuracy,Precision,Recall,F1
related,0.989319,0.989319,0.989319,0.989319
request,0.987489,0.987489,0.987489,0.987489
offer,0.998779,0.998779,0.998779,0.998779
aid_related,0.984183,0.984183,0.984183,0.984183
medical_help,0.989014,0.989014,0.989014,0.989014
medical_products,0.991862,0.991862,0.991862,0.991862
search_and_rescue,0.993999,0.993999,0.993999,0.993999
security,0.994965,0.994965,0.994965,0.994965
military,0.996135,0.996135,0.996135,0.996135
child_alone,1.000000,1.000000,1.000000,1.000000


In [16]:
model_impro.best_params_

{'clf__estimator__min_samples_split': 2, 'tfidf__smooth_idf': True}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
#Use a different estimator (DTC) to try and improve the model further
from sklearn.tree import DecisionTreeClassifier
pipeline_dtc = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()), 
    ('clf', MultiOutputClassifier(DecisionTreeClassifier())),
])

In [20]:
#hyper-parameter selection 
dtc_params = {
    'tfidf__smooth_idf':[False],
    #'clf__estimator__degree':[2]
}

cv_dtc = GridSearchCV(pipeline_dtc, dtc_params, cv=3, n_jobs=-1)
cv_dtc

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__smooth_idf': [False]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [21]:
dtc_fit = cv_dtc.fit(X_train, y_train)

In [23]:
dtc_pred = dtc_fit.predict(X_train)

In [24]:
dtc_model = pred_loop(np.array(y_train), dtc_pred, col_names)

In [25]:
dtc_model

,Accuracy,Precision,Recall,F1
related,0.998067,0.998067,0.998067,0.998067
request,0.999390,0.999390,0.999390,0.999390
offer,0.999949,0.999949,0.999949,0.999949
aid_related,0.999135,0.999135,0.999135,0.999135
medical_help,0.999695,0.999695,0.999695,0.999695
medical_products,0.999847,0.999847,0.999847,0.999847
search_and_rescue,0.999949,0.999949,0.999949,0.999949
security,0.999898,0.999898,0.999898,0.999898
military,0.999898,0.999898,0.999898,0.999898
child_alone,1.000000,1.000000,1.000000,1.000000


### 9. Export your model as a pickle file

In [26]:
pickle.dump(cv, open('ml_model.p', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.